In [8]:
### SPARK imports

from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [9]:
lg = LinearRegression()
pipe = Pipeline(stages=[lg])




In [12]:
paramGrid = (ParamGridBuilder()
             .baseOn({lg.maxIter:10})
             .baseOn({lg.predictionCol:"prediction"})
             .baseOn({lg.featuresCol:"features"})
             .baseOn({lg.labelCol:"label"})
             .addGrid(lg.elasticNetParam, [0.1, 0.01])
             .addGrid(lg.regParam,[0.99,0.1,0.01])
             .build())


crossval = CrossValidator(estimator=pipe,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=10)  # use 3+ folds in practice

In [ ]:
#import data and do transformations


In [13]:
def executeLinearRegressions(df,targetCol,crossval):
    """
        this method should do linear regression for 
        each of the companies or another column that we can chose. 
    """
    
    #Extract unqiue cvr-numbers
    uniqueTargetValues = (df
                          .select(targetCol)
                          .distinct()
                          .rdd
                          .map(lambda x: x[0])
                          .collect()
                         )
    modelsPrCvr = {} # add all models with cvr numbers in this one.
    
    #Run through all cvr-numbers 
    for company in uniqueTargetValues:
        tempDf = df.filter(F.col(targetCol) == company)
        
        cvModel = crossval.fit(tempDf)
        modelsPrCvr[company] = cvModel
    
    
    return modelsPrCvr
    
    

In [ ]:
# Run cross-validation, and choose the best set of parameters.
allModels = executeLinearRegressions(,,crossval)